<a href="https://colab.research.google.com/github/Imuseless123/InventoryOP/blob/main/Colaboratory_ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np

# Load your data with semicolon as the separator and set low_memory to False to avoid mixed-type warnings
df = pd.read_csv('/content/sample_data/DataCoSupplyChainDataset.csv', encoding='latin-1', sep=';', low_memory=False)
print(df);


            Type  Days for shipping (real)  Days for shipment (scheduled)  \
0          DEBIT                         3                              4   
1       TRANSFER                         5                              4   
2           CASH                         4                              4   
3          DEBIT                         3                              4   
4        PAYMENT                         2                              4   
...          ...                       ...                            ...   
180514      CASH                         4                              4   
180515     DEBIT                         3                              2   
180516  TRANSFER                         5                              4   
180517   PAYMENT                         3                              4   
180518   PAYMENT                         4                              4   

       Benefit per order Sales per customer   Delivery Status  \
0         

In [12]:
# Step 1: Handle Missing Values
# Fill numerical columns with median values and categorical columns with "Unknown"
numerical_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(exclude=[np.number]).columns

# Convert columns with periods or commas into numeric format by removing non-numeric characters
df['Sales per customer'] = pd.to_numeric(df['Sales per customer'].str.replace('.', ''), errors='coerce')
df['Product Price'] = pd.to_numeric(df['Product Price'].str.replace('.', ''), errors='coerce')
df['Benefit per order'] = pd.to_numeric(df['Benefit per order'].str.replace('.', ''), errors='coerce')

# Fill NaNs in numerical columns with the median and categorical columns with "Unknown"
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

In [13]:
# Step 2: Remove Duplicates
df.drop_duplicates(inplace=True)


In [14]:
# Step 3: Correct Data Types
# Convert date columns to datetime format, handling any parsing errors
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')
df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'], errors='coerce')

<ipython-input-14-ad39e12decdf>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['shipping date (DateOrders)'] = pd.to_datetime(df['shipping date (DateOrders)'], errors='coerce')


In [ ]:
# Step 4: Validate Numerical Values
# Ensure no negative or invalid values in key columns
df['Order Item Product Price'] = df['Order Item Product Price'].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
df['Sales per customer'] = df['Sales per customer'].apply(lambda x: max(x, 0) if pd.notnull(x) else x)

In [ ]:
# Step 5: Standardize Categorical Data
# Capitalize categorical fields for consistency
df['Delivery Status'] = df['Delivery Status'].str.title()
df['Product Status'] = df['Product Status'].str.title()

In [ ]:
# Step 6: Format Text Fields
# Remove extra whitespace and standardize name fields
df['Customer Fname'] = df['Customer Fname'].str.strip().str.title()
df['Customer Lname'] = df['Customer Lname'].str.strip().str.title()
df['Customer Email'] = df['Customer Email'].str.strip()

In [ ]:
# Step 7: Verify Geolocation Data
df = df[(df['Latitude'] >= -90) & (df['Latitude'] <= 90)]
df = df[(df['Longitude'] >= -180) & (df['Longitude'] <= 180)]

In [ ]:
# Step 8: Calculate Derived Columns
df['Actual Delivery Time'] = (df['shipping date (DateOrders)'] - df['order date (DateOrders)']).dt.days
df['Late Delivery'] = np.where(df['Actual Delivery Time'] > df['Days for shipment (scheduled)'], 1, 0)

In [ ]:
# Step 9: Remove Irrelevant or Sensitive Columns
df.drop(columns=['Customer Password'], inplace=True)

In [ ]:
# Save cleaned data to a new CSV
df.to_csv('cleaned_data.csv', index=False)